# Project 4

In this project, you will summarize and present your analysis from Projects 1-3.

### Intro: Write a problem Statement/ Specific Aim for this project

Answer: This analysis determines how GPA, GRE, and Prestige of previous school impact admittance.

### Dataset:  Write up a description of your data and any cleaning that was completed

Answer: The dataset includes three columns for the independent variables: GPA (weighted on a 4.0 scale), GRE Score, Prestige of previous school on a scale of 1 to 4. The dataset also has a column that has a 0 if the person was not admitted and a 1 if they were admitted. The dataset had some missing values which were dropped for the analysis.

### Demo: Provide a table that explains the data by admission status

Mean (STD) or counts by admission status for each variable 

| Not Admitted | Admitted
---| ---|---
GPA | mean(std)  | mean(std)
GRE |mean(std) | mean(std)
Prestige 1 | frequency (%) | frequency (%)
Prestige 2 | frequency (%) | frequency (%)
Prestige 3 |frequency (%) | frequency (%)
Prestige 4 |frequency (%) | frequency (%)

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np

In [3]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 

In [39]:
prestigeadmit = pd.crosstab(index = df["prestige"], columns = df["admit"], normalize ="index")

In [40]:
dfdescribe = df.groupby("admit").describe()

In [41]:
descriptiontable = pd.DataFrame([[3.35, 3.49], [574, 619], [0.46, 0.54], [0.64, 0.36], [0.77, 0.23], [0.82, 0.18]], index = ["gpa mean", "gre mean", "prestige 1", "prestige 2", "prestige 3", "prestige 4"], columns = ["not admitted", "admitted"])
descriptiontable

,not admitted,admitted
gpa mean,3.35,3.49
gre mean,574.00,619.00
prestige 1,0.46,0.54
prestige 2,0.64,0.36
prestige 3,0.77,0.23
prestige 4,0.82,0.18


### Methods: Write up the methods used in your analysis

For this analysis, we used a Logistic Regression to determine the probabilities of someone being admitted with various combinations of GPA, GRE, and Prestige. 

### Results: Write up your results

Answer: Our analysis showed that people who had higher GPA scores, GRE scores, and went to more prestigious schools were more likely to be admitted.

### Visuals: Provide a table or visualization of these results

In [4]:
dummy_ranks = pd.get_dummies(df["prestige"], prefix = "prestige")
dummy_ranks.head()

,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,0,1,0
1,0,0,1,0
2,1,0,0,0
3,0,0,0,1
4,0,0,0,1


In [5]:
cols = ["admit", "gre", "gpa"]

In [7]:
data = df[cols].join(dummy_ranks.ix[:, "prestige_2":])
data.head()

,admit,gre,gpa,prestige_2.0,prestige_3.0,prestige_4.0
0,0,380.0,3.61,0,1,0
1,1,660.0,3.67,0,1,0
2,1,800.0,4.00,0,0,0
3,1,640.0,3.19,0,0,1
4,0,520.0,2.93,0,0,1


In [9]:
data["intercept"] = 1.0

In [23]:
train_cols = data[["gre", "gpa", "prestige_2.0", "prestige_3.0", "prestige_4.0"]
train_cols

SyntaxError: invalid syntax (<ipython-input-23-4b707a6e1132>, line 2)

In [11]:
logit = sm.Logit(data["admit"], data[train_cols])

In [12]:
result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


In [13]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Thu, 01 Jun 2017   Pseudo R-squ.:                 0.08166
Time:                        18:28:02   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
================================================================================
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa              0.7793      0.333      2.344      0.019         0.128     1.431
prestige_2.0    -0.6801      0.317     -2.146      0.032        -1.301    -0.059
prestige_3.0    -1.3387      0.345     -3.882      0.000        -2.015    -0.663
prestige_4.0    -1.5534      0.417     -3.721      0.000        -2.372    -0.735
intercept       -3.8769      1.142     -3.393      0.001        -6.116    -1.638
================================================================================
"""

In [14]:
def cartesian(arrays, out=None):
 
    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [15]:
gres = np.linspace(df['gre'].min(), df['gre'].max(), 10)
print gres

gpas = np.linspace(df['gpa'].min(), df['gpa'].max(), 10)
print gpas

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


In [16]:
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

In [17]:
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
dummy_ranks = pd.get_dummies(combos['prestige'], prefix='prestige')
dummy_ranks.columns = ['prestige_1', 'prestige_2', 'prestige_3', 'prestige_4']

In [18]:
dummy_ranks.head()

,prestige_1,prestige_2,prestige_3,prestige_4
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0


In [19]:
cols_to_keep = ['gre', 'gpa', 'prestige', 'intercept']
combos = combos[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2.0':])

In [20]:
combos.head()

,gre,gpa,prestige,intercept,prestige_3,prestige_4
0,220.0,2.260000,1.0,1.0,0,0
1,220.0,2.260000,2.0,1.0,0,0
2,220.0,2.260000,3.0,1.0,1,0
3,220.0,2.260000,4.0,1.0,0,1
4,220.0,2.453333,1.0,1.0,0,0


In [21]:
combos['admit_pred'] = result.predict(combos[train_cols])

KeyError: "Index([u'prestige_2.0', u'prestige_3.0', u'prestige_4.0'], dtype='object') not in index"

In [69]:
combos.head()

,gre,gpa,prestige,intercept,prestige_2,prestige_3,prestige_4
0,220.0,2.260000,1.0,1.0,0,0,0
1,220.0,2.260000,2.0,1.0,1,0,0
2,220.0,2.260000,3.0,1.0,0,1,0
3,220.0,2.260000,4.0,1.0,0,0,1
4,220.0,2.453333,1.0,1.0,0,0,0


In [43]:
  def isolate_and_plot(variable):
      grouped = pd.pivot_table(combos, values=['admit_pred'], index=[variable, 'prestige'],
                  aggfunc=np.mean)
   
      colors = 'rbgyrbgy'
      for col in combos.prestige.unique():
          plt_data = grouped.ix[grouped.index.get_level_values(1)==col]
          pl.plot(plt_data.index.get_level_values(0), plt_data['admit_pred'], color=colors[int(col)])

      pl.xlabel(variable)
      pl.ylabel("P(admit=1)")
      pl.legend(['1', '2', '3', '4'], loc='upper left', title='Prestige')
      pl.title("Prob(admit=1) isolating " + variable + " and presitge")
      pl.show()

  isolate_and_plot('gre')
  isolate_and_plot('gpa')

KeyError: 'prestige'

### Discussion: Write up your discussion and future steps

Answer: I would experiment with running other types of models on this dataset, such as a Random Forest. With a Random Forest I could potentially determine which features are the most "influential" on the outcome variable and use this to build a new model.